# PoroDet: Nanoporosity Detection using Google Colab
This notebook demonstrates how to use the PoroDet Python package to train a U-Net model, detect nanopores in new images, and analyze the results using Google Drive.
Note: A user may use the local storage to upload the data. But it will be preferable to use the google drive for mutiple time access.

**Step 1: Install PoroDet**
Install the package directly from GitHub. This ensures we are using the latest version of the code.
Install the Augmentations lastest version if possible.



In [ ]:
# Install the PoroDet package directly from GitHub
!pip install git+https://github.com/Deep7285/PoroDet.git

# Install other dependencies if they aren't caught automatically
!pip install albumentations==1.3.1  # Ensuring compatible version

  Cloning https://github.com/Deep7285/PoroDet.git to /tmp/pip-req-build-3regzd83
  Running command git clone --filter=blob:none --quiet https://github.com/Deep7285/PoroDet.git /tmp/pip-req-build-3regzd83
  Resolved https://github.com/Deep7285/PoroDet.git to commit f35bb0dc6cfcc6672906639af27ae3bece6a45a0
  Preparing metadata (setup.py) ... done
  Created wheel for porodet: filename=porodet-0.1.0-py3-none-any.whl size=22476 sha256=33461bb1343b64908ef58b5a52ab1978d564f12d7abfa0eca07eee17e36c061c
  Stored in directory: /tmp/pip-ephem-wheel-cache-zzz1fsrv/wheels/0b/9f/2b/7b5165f50c852e599881d3c840f13497b37841a1103cfa77c7
Successfully built porodet
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 2.0.8
    Uninstalling albumentations-2.0.8:
      Successfully uninstalled albumentations-2.0.8


**Step 2: Mount Google Drive or upload the input data**
We need access to your dataset and a place to save the trained model.
Note: keep both raw '.tif' file and its corrresponding binary mask. A sample will can be found in readme.md

In [ ]:
# If data availaable on google drive then choose this
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# --- CONFIGURATION ---
# Define your paths here
TRAIN_DATA_DIR = "/content/drive/MyDrive/Rajat_Zr_nanoporosity/Masked_Dataset/Augmented_data"
INFERENCE_DATA_DIR = "/content/drive/MyDrive/Rajat_Zr_nanoporosity/Masked_Dataset/New_Inference_Images"
OUTPUT_DIR = "/content/drive/MyDrive/Rajat_Zr_nanoporosity/package_testing_colab"

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Training Data: {TRAIN_DATA_DIR}")
print(f"Inference Data: {INFERENCE_DATA_DIR}")
print(f"Output Folder: {OUTPUT_DIR}")

Mounted at /content/drive
Training Data: /content/drive/MyDrive/Rajat_Zr_nanoporosity/Masked_Dataset/Augmented_data
Inference Data: /content/drive/MyDrive/Rajat_Zr_nanoporosity/Masked_Dataset/New_Inference_Images
Output Folder: /content/drive/MyDrive/Rajat_Zr_nanoporosity/package_testing_colab


In [ ]:
# if data available on local machine or folder then use this to upload
import os
from google.colab import files
import zipfile

# Upload the file
print("Please upload your dataset as a ZIP file (e.g., data.zip)")
uploaded = files.upload()

# Get the filename of the uploaded zip
zip_filename = next(iter(uploaded))
print(f"Uploaded: {zip_filename}")

# Extract the Zip file
extract_path = "/content/dataset"
os.makedirs(extract_path, exist_ok=True)

print("Extracting files")
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Extraction complete. Data is in: {extract_path}")

#  Update paths for dataset uploadeing
# (Name the zip files contains folders named 'Augmented_data' and 'New_Inference_Images')
# You might need to adjust these based on how users zip their folders
TRAIN_DATA_DIR = os.path.join(extract_path, "Augmented_data")
INFERENCE_DATA_DIR = os.path.join(extract_path, "New_Inference_Images")
OUTPUT_DIR = "/content/porodet_output"

os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"\n--- PATHS UPDATED ---")
print(f"Training Data: {TRAIN_DATA_DIR}")
print(f"Inference Data: {INFERENCE_DATA_DIR}")
print(f"Output Folder: {OUTPUT_DIR}")

**Step 3: Train the Model**
The porodet.train function expects a list of specific image files for training and validation. Since we are in a notebook (and can't use pop-up windows), we will write a small script to:
1. Scan the data folder.
2. Group original images with their augmented versions.
3. Split them into Training and Validation sets.
4. Run the training.

In [ ]:
import porodet
import numpy as np
import torch
from sklearn.model_selection import KFold
from porodet.training import get_original_and_augmented_groups

# Check what hardware we are running on, code will check the hardware and process the data accordingly.
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    total_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"Found GPU: {gpu_name} ({total_mem:.2f} GB VRAM)")

    if total_mem < 20:
        # For T4 (Free Colab) -> 15GB VRAM
        print("⚠️ Detected Standard GPU. Reducing image size to 512x512 to prevent crash.")
        TARGET_SIZE = (512, 512)
        BATCH_SIZE = 2 # 512x512 is small enough to run batch=2 safely
    else:
        # For A100 (Pro Colab) -> 40GB+ VRAM
        print("🚀 Detected High-End GPU. Using full 1024x1024 resolution.")
        TARGET_SIZE = (1024, 1024)
        BATCH_SIZE = 2 # change the batch size as per hardware compatibilty
else:
    print("❌ No GPU found! Training will be extremely slow.")
    TARGET_SIZE = (256, 256) # Emergency mode for CPU
    BATCH_SIZE = 1

# Group images
print("\nGrouping files...")
original_groups = get_original_and_augmented_groups(TRAIN_DATA_DIR)
original_images = list(original_groups.keys())
print(f"Found {len(original_images)} unique original images.")

# Create Train/Val split
kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_indices, val_indices = next(kf.split(original_images))

train_originals = [original_images[i] for i in train_indices]
val_originals = [original_images[i] for i in val_indices]

# Define Training Output Folder where training logs and trained model will be saved
training_output = os.path.join(OUTPUT_DIR, "Training_Session")
os.makedirs(training_output, exist_ok=True)

# 4. Start Training
print(f"\n--- Starting model training ({TARGET_SIZE[0]}x{TARGET_SIZE[1]}) ---")

# We pass 'resize_to' to override the default 1024 size
model, history = porodet.train(
    data_dir=TRAIN_DATA_DIR,
    output_dir=training_output,
    train_originals=train_originals,
    val_originals=val_originals,
    batch_size=BATCH_SIZE,
    resize_to=TARGET_SIZE,
    epochs=25,  # optimize the epoch value to get the loss plateau
    learning_rate=5e-5, # adjust the learning rate to optimization
    patience=3,
    fold_id=1  #adjust the fold id. Idealy keep 3.
)

print("Training Complete!")

Found GPU: Tesla T4 (15.83 GB VRAM)
⚠️ Detected Standard GPU. Reducing image size to 512x512 to prevent crash.

Grouping files...
Found 26 unique original images.

--- Starting PoroDet Training (512x512) ---

Fold 1: Using device cuda
Fold 1: 20 originals for training, 6 for validation
Fold 1: Validation originals: ['Image_1', 'Image_17', 'Image_18', 'Image_2', 'Image_26', 'Image_8']
Training set contains 320 images
Validation set contains 96 images

Starting training with verbose metrics to monitor progress
Early stopping patience: 3
Learning rate: 5e-05
Weight decay: 0.0001


Fold 1 | Epoch 1/25 [train]: 100%|██████████| 160/160 [07:02<00:00,  2.64s/it, loss=0.3419]


Fold 1 & Epoch 1: Train Loss 0.4614, Train Acc 0.9310 | Val Loss 0.3733, Val Acc 0.9762 | Val Prec 0.5203, Rec 0.2211, F1 0.3103, IoU 0.1837PR-AUC 0.24046968272141384, ROC-AUC 0.8434281466263029
Fold 1: Saved new best model (val loss 0.3733)


Fold 1 | Epoch 2/25 [train]: 100%|██████████| 160/160 [02:54<00:00,  1.09s/it, loss=0.2818]


Fold 1 & Epoch 2: Train Loss 0.3260, Train Acc 0.9776 | Val Loss 0.2927, Val Acc 0.9786 | Val Prec 0.8602, Rec 0.1357, F1 0.2344, IoU 0.1327PR-AUC 0.3364830362434005, ROC-AUC 0.8411680153634721
Fold 1: Saved new best model (val loss 0.2927)


...

Fold 1 & Epoch 23: Train Loss 0.0603, Train Acc 0.9839 | Val Loss 0.0640, Val Acc 0.9822 | Val Prec 0.6728, Rec 0.5164, F1 0.5843, IoU 0.4128PR-AUC 0.6147420904433771, ROC-AUC 0.9642315564932835
Fold 1: Validation loss did not improve. Counter: 1/3


Fold 1 | Epoch 24/25 [train]: 100%|██████████| 160/160 [02:54<00:00,  1.09s/it, loss=0.0680]


Fold 1 & Epoch 24: Train Loss 0.0583, Train Acc 0.9841 | Val Loss 0.0620, Val Acc 0.9824 | Val Prec 0.6544, Rec 0.5771, F1 0.6133, IoU 0.4423PR-AUC 0.6572395446143184, ROC-AUC 0.9631158348288398
Fold 1: Saved new best model (val loss 0.0620)


Fold 1 | Epoch 25/25 [train]: 100%|██████████| 160/160 [03:03<00:00,  1.15s/it, loss=0.0520]


Fold 1 & Epoch 25: Train Loss 0.0558, Train Acc 0.9843 | Val Loss 0.0586, Val Acc 0.9837 | Val Prec 0.7398, Rec 0.5033, F1 0.5991, IoU 0.4276PR-AUC 0.665311058874434, ROC-AUC 0.9673061395461103
Fold 1: Saved new best model (val loss 0.0586)
Fold 1: Training finished. Final model saved to: /content/drive/MyDrive/Rajat_Zr_nanoporosity/package_testing_colab/Training_Session/fold_1_final_model.pth
Fold 1: Saved metrics CSV to: /content/drive/MyDrive/Rajat_Zr_nanoporosity/package_testing_colab/Training_Session/fold_1_metrics.csv
Training Complete!


**Step 4: Detect Pores (Inference)**
Now we take the best model saved from the previous step and apply it to new, unseen images.

In [ ]:
import torch
import cv2
import matplotlib.pyplot as plt
from glob import glob

# Find the best model file from the training output
model_files = glob(os.path.join(training_output, "*_best_model.pth"))
if not model_files:
    raise FileNotFoundError("No trained model found!")
best_model_path = model_files[0]
print(f"Loading the Model: {best_model_path}")

# Load the Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = porodet.UNet(in_channels=1, out_channels=1).to(device)

# Added weights_only=False to fix PyTorch 2.6 compatibility
checkpoint = torch.load(best_model_path, map_location=device, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Run Inference on New Images
inference_images = sorted(glob(os.path.join(INFERENCE_DATA_DIR, "*.tif"))) # Adjust extension if needed (.jpg/.png)

if not inference_images:
    print("No images found in inference folder.")
else:
    print(f"Found {len(inference_images)} images for inference.")

    inference_output_dir = os.path.join(OUTPUT_DIR, "Inference_Results")
    os.makedirs(inference_output_dir, exist_ok=True)

    for img_path in inference_images:
        img_name = os.path.basename(img_path)
        print(f"Processing: {img_name}...")

        # Detection
        original, prob_map, binary_mask = porodet.detect(model, img_path, device, threshold=0.5)

        # Save the mask for the next step i.e Analysis
        mask_filename = os.path.splitext(img_name)[0] + "_mask.png"
        mask_save_path = os.path.join(inference_output_dir, mask_filename)
        cv2.imwrite(mask_save_path, (binary_mask * 255).astype(np.uint8))

        # Optional: Save visualization overlay
        # package's visualization tools or simple matplotlib can be used)
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1); plt.imshow(original, cmap='gray'); plt.title("Original")
        plt.subplot(1, 2, 2); plt.imshow(prob_map, cmap='hot'); plt.title("Prediction")
        plt.savefig(os.path.join(inference_output_dir, f"{img_name}_vis.png"))
        plt.close()

    print(f"\nInference done. Results saved to: {inference_output_dir}")

Loading Model: /content/drive/MyDrive/Rajat_Zr_nanoporosity/package_testing_colab/Training_Session/fold_1_best_model.pth
Found 1 images for inference.
Processing: Image_19.tif...

Inference done. Results saved to: /content/drive/MyDrive/Rajat_Zr_nanoporosity/package_testing_colab/Inference_Results


**Step 5: Analyze Results**
Finally, we calculate porosity statistics (area, pore count, shape) using the masks generated in the previous step.

In [ ]:
# Get the list of generated masks
mask_files = sorted(glob(os.path.join(inference_output_dir, "*_mask.png")))

if not mask_files:
    print("No masks found to analyze.")
else:
    analysis_output_dir = os.path.join(OUTPUT_DIR, "Analysis_Report")
    os.makedirs(analysis_output_dir, exist_ok=True)

    print(f"Analyzing {len(mask_files)} masks...")

    for mask_path in mask_files:
        print(f"Analyzing: {os.path.basename(mask_path)}")

        # Analysis
        # porodet.analyze takes the mask path and output directory
        porodet.analyze(
            mask_path=mask_path,
            output_dir=analysis_output_dir,
            aspect_ratio_threshold=2.0 # Threshold to distinguish pores vs cracks, adjust it accordingly
        )

    print(f"\nAnalysis Report saved to: {analysis_output_dir}")

Analyzing 1 masks...
Analyzing: Image_19_mask.png
Image_19.tif: Nanoporosity = 0.65%
Total number of nanopores detected: 211
Number of mostly circular nanopores (aspect ratio ≤ 2.0): 176
Number of elongated nanopores (aspect ratio > 2.0): 35
Average size: 32.42 pixels
Average aspect ratio: 1.57
Average diameter: 5.88 pixels
Size histogram saved to: /content/drive/MyDrive/Rajat_Zr_nanoporosity/package_testing_colab/Analysis_Report/Image_19_size_histogram.png
Aspect ratio histogram saved to: /content/drive/MyDrive/Rajat_Zr_nanoporosity/package_testing_colab/Analysis_Report/Image_19_aspect_ratio_histogram.png
Diameter histogram saved to: /content/drive/MyDrive/Rajat_Zr_nanoporosity/package_testing_colab/Analysis_Report/Image_19_diameter_histogram.png
Circular overlay saved to: /content/drive/MyDrive/Rajat_Zr_nanoporosity/package_testing_colab/Analysis_Report/Image_19_circular_overlay.png
Elongated overlay saved to: /content/drive/MyDrive/Rajat_Zr_nanoporosity/package_testing_colab/Analysi